# Get Online Inference from Vertex AI Endpoint - Example Template

This notebook demonstrates how to get predictions from a deployed Vertex AI Pipeline Endpoint.
**This is an example template - copy and configure for your use.**

## 🔐 Setup Instructions:

1. Copy this notebook and remove `.example` from the filename
2. Configure your GCP project using `config.py` or direct configuration
3. Make sure you deployed the pipeline using `07_Pipeline_Deployment/`
4. Run the cells below


In [ ]:
# Import Required Libraries
from google.cloud import aiplatform

print("Libraries imported successfully.")

In [ ]:
# Authenticate with Google Cloud
import google.auth

try:
    # Try to get default credentials
    credentials, project = google.auth.default()
    print("✅ Using existing credentials")
except:
    # If no credentials found, use gcloud auth
    print("⚠️ No credentials found. Please authenticate using one of these methods:\n")
    print("Method 1 (Recommended for local development):")
    print("   Run in terminal: gcloud auth application-default login\n")
    print("Method 2 (For Colab):")
    print("   from google.colab import auth")
    print("   auth.authenticate_user()\n")
    print("Method 3 (Service Account):")
    print("   Set environment variable: GOOGLE_APPLICATION_CREDENTIALS=path/to/key.json")

## Configuration

**Option 1: Use .env file (Recommended)**

1. Copy `.env.example` to `.env` in **this directory** (08_Hit_Pipeline_Deployment/)
2. Edit `.env` with your GCP project details
3. Run the config cell below

**Option 2: Direct Configuration**

Fill in values directly in the second cell


In [ ]:
# Option 1: Load from local config.py (which loads from .env)
try:
    from config import GCP_PROJECT_ID, GCP_REGION, ENDPOINT_DISPLAY_NAME

    PROJECT_ID = GCP_PROJECT_ID
    REGION = GCP_REGION

    print("✅ Configuration loaded from config.py")
    print(f"Project: {PROJECT_ID}")
    print(f"Region: {REGION}")
    print(f"Endpoint Name: {ENDPOINT_DISPLAY_NAME}")

except ImportError:
    print("⚠️ config.py not found in this directory.")
    print("Please create .env file (copy from .env.example) or use Option 2 below.")
    PROJECT_ID = None
    REGION = None
    ENDPOINT_DISPLAY_NAME = None

In [ ]:
# Option 2: Direct Configuration (uncomment and fill in)
# Uncomment these lines if not using config.py

# PROJECT_ID = "your-project-id-here"  # Your GCP Project ID
# REGION = "us-central1"  # Your GCP region (e.g., us-central1, asia-southeast1)
# ENDPOINT_DISPLAY_NAME = "Pipeline_Deployment_Model_Endpoint"  # Your endpoint name

# Print configuration
if PROJECT_ID and PROJECT_ID not in ["YOUR_PROJECT_ID", "your-project-id-here"]:
    print(f"Project: {PROJECT_ID}")
    print(f"Region: {REGION}")
    print(f"Endpoint Name: {ENDPOINT_DISPLAY_NAME}")
else:
    print("❌ Please configure PROJECT_ID, REGION, and ENDPOINT_DISPLAY_NAME")

## Initialize Vertex AI SDK


In [ ]:
# Initialize Vertex AI
if PROJECT_ID and PROJECT_ID not in ["YOUR_PROJECT_ID", "your-project-id-here"]:
    aiplatform.init(project=PROJECT_ID, location=REGION)
    print("✅ Vertex AI SDK Initialized.")
else:
    print("❌ Please configure your project first")

## Get Endpoint Object


In [ ]:
# Get Endpoint Object
if PROJECT_ID and PROJECT_ID not in ["YOUR_PROJECT_ID", "your-project-id-here"]:
    try:
        endpoints = aiplatform.Endpoint.list(filter=f'display_name="{ENDPOINT_DISPLAY_NAME}"')

        if endpoints:
            endpoint = endpoints[0]
            print(f"✅ Found Endpoint: {endpoint.resource_name}")
        else:
            print(f"❌ Endpoint '{ENDPOINT_DISPLAY_NAME}' not found in region {REGION}.")
            print("Please check the name and region.")
            print("\nTip: List all endpoints with:")
            print("  endpoints = aiplatform.Endpoint.list()")
            print("  for ep in endpoints: print(ep.display_name)")
    except Exception as e:
        print(f"❌ Error accessing endpoint: {str(e)}")
        print("\nCommon issues:")
        print("1. Authentication: Run 'gcloud auth application-default login' in terminal")
        print("2. Project ID: Make sure you're using the project ID (name), not project number")
        print("3. Permissions: Ensure you have Vertex AI User role")
        print("4. API: Enable Vertex AI API in your project")
else:
    print("❌ Please configure your project first")

## 📌 Important: Pipeline Deployment

This notebook assumes you deployed the **complete Pipeline** (vectorizer + model).

If you deployed using `07_Pipeline_Deployment/Deploy_Pipeline_to_Vertex_AI.ipynb`,
you can send raw text directly to the endpoint.

**What's deployed:**

- ✅ Pipeline (vectorizer + model) → Send raw text: `["your review text"]`
- ❌ Model only → Need to vectorize locally first


## Test with Raw Text (Pipeline Deployment)


In [ ]:
# Test: Send raw text (works if pipeline was deployed)
if 'endpoint' in locals():
    new_review = "I recently purchased this dress and I have to say, it exceeded my expectations!"
    print(f"Testing with: '{new_review}'")
    print("\n...Calling the endpoint with raw text...")

    try:
        response = endpoint.predict(instances=[new_review])
        print("\n✅ SUCCESS! Pipeline deployment works!")
        print(f"Raw prediction: {response.predictions[0]}")

        if response.predictions[0] == 1:
            print("📊 Sentiment: POSITIVE ✅")
        else:
            print("📊 Sentiment: NEGATIVE ❌")

    except Exception as e:
        print(f"\n❌ ERROR: {str(e)}")
        print("\nPossible issues:")
        print("1. Pipeline was not deployed (only model was deployed)")
        print("2. Wrong endpoint name")
        print("3. Model expects different input format")
else:
    print("❌ Endpoint not configured")

## Test Multiple Reviews


In [ ]:
# Test Multiple Reviews at Once
if 'endpoint' in locals():
    test_reviews = [
        "I absolutely love this dress! The fit is perfect and the material is so soft.",
        "The shirt shrunk after one wash. Very disappointed with the quality.",
        "Amazing product! Highly recommend to everyone!",
        "Terrible quality. Waste of money.",
        "It's okay, nothing special but not bad either."
    ]

    print("Testing multiple reviews...\n")
    print("=" * 80)

    try:
        for i, review in enumerate(test_reviews, 1):
            response = endpoint.predict(instances=[review])
            prediction = response.predictions[0]
            sentiment = "POSITIVE ✅" if prediction == 1 else "NEGATIVE ❌"

            print(f"\n{i}. Review: {review}")
            print(f"   Prediction: {prediction} → {sentiment}")

        print("\n" + "=" * 80)

    except Exception as e:
        print(f"❌ Error: {str(e)}")
else:
    print("❌ Endpoint not configured")

---

## 🎉 Pipeline Deployment Benefits

**Pipeline Deployment වාසි:**

✅ **No Vectorizer needed locally** - Pipeline එකේම තියෙනවා  
✅ **Send raw text directly** - `endpoint.predict(instances=["text"])`  
✅ **Version control** - Vectorizer සහ Model එකම එක්ක update වෙනවා  
✅ **Production ready** - Clean, maintainable code

---

## 📝 Quick Reference

```python
# Single prediction
response = endpoint.predict(instances=["Your review text here"])
print(response.predictions[0])  # 1 = Positive, -1 = Negative

# Multiple predictions
reviews = ["review 1", "review 2", "review 3"]
response = endpoint.predict(instances=reviews)
print(response.predictions)  # [1, -1, 1]
```

---

## 🔗 Related Notebooks

- **Training & Pipeline Creation:** [07_Pipeline_Deployment/Linear_Classificaton - Logistic Regression Model.ipynb](../07_Pipeline_Deployment/Linear_Classificaton%20-%20Logistic%20Regression%20Model%20.ipynb)
- **Deployment Guide:** [07_Pipeline_Deployment/Deploy_Pipeline_to_Vertex_AI.ipynb](../07_Pipeline_Deployment/Deploy_Pipeline_to_Vertex_AI.ipynb)
- **Example Template:** [06_Get_Online_Inference/06_Get_Online_Inference.example.ipynb](../06_Get_Online_Inference/06_Get_Online_Inference.example.ipynb)
